Reference: https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mapgccv/miscellaneous/sudoku.ipynb#scrollTo=jxi-HR24j7jh

In [1]:
%pip install -q amplpy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00


In [5]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [21]:
%%ampl_eval
reset;
option solver highs;

param BASE default 3;
param L := BASE*BASE;

set ROWS := {1..L};
set COLS := {1..L};
set SUBSQUARES{sr in 1..BASE, sc in 1..BASE} within {ROWS, COLS}
	            = {(sr-1)*BASE+1..sr*BASE, (sc-1)*BASE+1..sc*BASE};

param givenData{ROWS, COLS} default 0;

var x{ROWS, COLS} >=1, <=L integer;

# Dummy objective, just to "encourage" the solver to get the same
# objective function in case of a degenerate sudoku
maximize z: x[1,1];

# Fix input data
subject to fixGivenData {r in ROWS, c in COLS : givenData[r,c] > 0}:
  x[r,c] = givenData[r,c];

In [22]:
ampl.display("SUBSQUARES")

set SUBSQUARES[1,1] := (1,1) (1,2) (1,3) (2,1) (2,2) (2,3) (3,1) (3,2) (3,3);
set SUBSQUARES[1,2] := (1,4) (1,5) (1,6) (2,4) (2,5) (2,6) (3,4) (3,5) (3,6);
set SUBSQUARES[1,3] := (1,7) (1,8) (1,9) (2,7) (2,8) (2,9) (3,7) (3,8) (3,9);
set SUBSQUARES[2,1] := (4,1) (4,2) (4,3) (5,1) (5,2) (5,3) (6,1) (6,2) (6,3);
set SUBSQUARES[2,2] := (4,4) (4,5) (4,6) (5,4) (5,5) (5,6) (6,4) (6,5) (6,6);
set SUBSQUARES[2,3] := (4,7) (4,8) (4,9) (5,7) (5,8) (5,9) (6,7) (6,8) (6,9);
set SUBSQUARES[3,1] := (7,1) (7,2) (7,3) (8,1) (8,2) (8,3) (9,1) (9,2) (9,3);
set SUBSQUARES[3,2] := (7,4) (7,5) (7,6) (8,4) (8,5) (8,6) (9,4) (9,5) (9,6);
set SUBSQUARES[3,3] := (7,7) (7,8) (7,9) (8,7) (8,8) (8,9) (9,7) (9,8) (9,9);



In [23]:
%%ampl_eval

# IsN is used for MIP formulation.
var IsN{1..L, COLS, ROWS} binary;

# Each position only one number
subject to MIPOnlyOneNumber {r in ROWS, c in COLS}:
  sum{n in 1..L} IsN[n,c,r] = 1;

# Link to the logical model variable
subject to MIPLinkToX {r in ROWS, c in COLS}:
  sum{n in 1..L} IsN[n,c,r]*n =x[r,c];


# Each number must be present in each row once
  # MIP formulation:
subject to MIPEachRowOneNumber {r in ROWS, n in 1..L}:
  sum{c in COLS} IsN[n,c,r] = 1;

  # CP formulation:
subject to rowsAllDiff{r in ROWS}:
  alldiff{c in COLS} x[r,c];

# Each number must be present in each col once
  # MIP formulation:
subject to MIPEachColOneNumber {r in COLS, n in 1..L}:
  sum{c in ROWS} IsN[n,c,r] = 1;

  # CP formulation:
subject to colsAllDiff{c in COLS}:
  alldiff{r in ROWS} x[r,c];

# Each number must be present in each subsquare once
  # MIP formulation:
subject to MIPEachSquareOneNumber {n in 1..L, sr in 1..BASE, sc in 1..BASE}:
  sum{(r, c) in SUBSQUARES[sr, sc]} IsN[n, c, r] = 1;

  # CP formulation:
squaresAllDiff{sr in 1..BASE, sc in 1..BASE}:
  alldiff{(c,r) in SUBSQUARES[sr,sc]} x[r,c];


# Define named problems to quickly switch between formulations
problem sudokuMIP: x, z, fixGivenData, IsN, MIPLinkToX, MIPOnlyOneNumber, MIPEachRowOneNumber, MIPEachColOneNumber, MIPEachSquareOneNumber;
problem sudokuCP: x, z, fixGivenData, rowsAllDiff, colsAllDiff, squaresAllDiff;

In [33]:
from random import seed, random

BASE = 3
# seed(1234)

def random_state():
    ampl.param["BASE"] = BASE
    if BASE != 3:
        return
    solution = [
        [2, 5, 7, 8, 6, 3, 1, 4, 9],
        [4, 9, 6, 5, 7, 1, 8, 3, 2],
        [8, 1, 3, 9, 4, 2, 7, 6, 5],
        [1, 6, 5, 2, 9, 4, 3, 7, 8],
        [9, 8, 4, 1, 3, 7, 5, 2, 6],
        [3, 7, 2, 6, 5, 8, 4, 9, 1],
        [7, 2, 9, 4, 8, 5, 6, 1, 3],
        [5, 3, 1, 7, 2, 6, 9, 8, 4],
        [6, 4, 8, 3, 1, 9, 2, 5, 7],
    ]
    ampl.param["givenData"] = {
        (i + 1, j + 1): solution[i][j] if random() <= 1 / 3.0 else 0
        for i in range(9)
        for j in range(9)
    }
    ampl.param["givenData"] = {(1, 1): 0}

random_state()

In [34]:
ampl.display("givenData")

givenData [*,*]
:   1   2   3   4   5   6   7   8   9    :=
1   0   0   0   0   0   0   1   0   9
2   4   0   0   0   0   1   0   3   2
3   8   0   3   0   0   2   0   6   5
4   1   0   5   0   0   0   0   0   0
5   0   0   0   1   0   7   0   0   0
6   0   0   0   6   0   0   0   9   1
7   7   0   0   0   8   5   0   0   3
8   0   0   0   0   0   0   0   0   0
9   0   4   8   0   0   9   2   0   7
;



In [35]:
problem_name = "sudokuMIP"
ampl.eval(f"solve {problem_name};")
ampl.display("x")

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 7
214 simplex iterations
1 branching nodes
 
x [*,*]
:   1   2   3   4   5   6   7   8   9    :=
1   7   5   2   8   6   3   1   4   9
2   4   9   6   7   5   1   8   3   2
3   8   1   3   9   4   2   7   6   5
4   1   2   5   3   9   4   7   8   6
5   8   6   9   1   5   7   4   3   2
6   4   3   7   6   8   2   5   9   1
7   7   6   9   2   8   5   4   1   3
8   2   5   3   1   7   4   9   8   6
9   1   4   8   6   3   9   2   5   7
;



In [36]:
problem_name = "sudokuCP"
ampl.eval(f"solve {problem_name};")
ampl.display("x")

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 5
107 simplex iterations
1 branching nodes
 
x [*,*]
:   1   2   3   4   5   6   7   8   9    :=
1   5   7   2   8   6   3   1   4   9
2   4   9   6   5   7   1   8   3   2
3   8   1   3   9   4   2   7   6   5
4   1   8   5   2   9   4   3   7   6
5   9   6   4   1   3   7   5   2   8
6   2   3   7   6   5   8   4   9   1
7   7   2   9   4   8   5   6   1   3
8   3   5   1   7   2   6   9   8   4
9   6   4   8   3   1   9   2   5   7
;



In [37]:
import ipywidgets as widgets
from IPython.display import display


class SudokuSchema:
    def _create_one_grid(self, startRow: int, startCol: int):
        gridItems = [
            widgets.VBox(
                [self.items[row, col] for row in range(startRow, startRow + self.BASE)]
            )
            for col in range(startCol, startCol + self.BASE)
        ]
        return widgets.HBox(
            gridItems, layout=widgets.Layout(border="solid 2px", width="140px")
        )

    def __init__(self, base):
        """Initializes a sudoku schema with base dimension BASE"""
        self.BASE = base
        self.BSQUARED = base**2
        # Create all widgets
        self.items = {
            (r, c): widgets.BoundedIntText(
                value=0,
                min=0,
                max=self.BASE**4,
                step=1,
                description="",
                layout=widgets.Layout(width="40px", height="40px"),
            )
            for r in range(self.BSQUARED)
            for c in range(self.BSQUARED)
        }
        self.sudoku = widgets.HBox(
            [
                widgets.VBox(
                    [
                        self._create_one_grid(r, c)
                        for r in range(0, self.BSQUARED, self.BASE)
                    ]
                )
                for c in range(0, self.BSQUARED, self.BASE)
            ]
        )
        self.create_selection_button()
        self.create_buttons()

    def display(self):
        """Display the current schema on the notebook"""
        display(self.sudoku)
        display(self.selector)
        if BASE == 3:
            display(widgets.HBox([self.random_button, self.solve_button]))
        else:
            display(widgets.HBox([self.solve_button]))

    def get_values(self):
        """Get the current non zero values as a (r,c) : value dictionary"""
        return {
            (r + 1, c + 1): self.items[r, c].value
            for r in range(self.BSQUARED)
            for c in range(self.BSQUARED)
            if self.items[r, c].value != 0
        }

    def set_values(self, values: dict):
        """Set the values in the schema from the specified (r,c) : value dictionary"""
        for (r, c), v in values.items():
            self.items[r - 1, c - 1].value = round(v)

    def create_selection_button(self):
        self.selector = widgets.RadioButtons(
            options=["Constraint Programming", "MIP"],
            value="Constraint Programming",
            layout={"width": "max-content"},
            description="Formulation:",
            disabled=False,
        )

    def get_selected_formulation(self):
        return self.selector.value

    def set_random_board(self):
        random_state()
        self.set_values(ampl.get_data("givenData").to_dict())

    def create_buttons(self):
        self.random_button = widgets.Button(
            description="Random board",
            disabled=False,
            button_style="info",
            tooltip="Random board",
            icon="bolt",
        )
        self.random_button.on_click(lambda btn: self.set_random_board())
        self.solve_button = widgets.Button(
            description="Solve",
            disabled=False,
            button_style="success",
            tooltip="Solve model",
            icon="bolt",
        )
        self.solve_button.on_click(solve_and_display)

In [38]:
# Solve and display the solution
def solve_and_display(button):
    # Get the selected formulation from the radio button
    formulation = sudoku.get_selected_formulation()
    if formulation == "Constraint Programming":
        problem_name = "sudokuCP"
    else:
        problem_name = "sudokuMIP"

    print(f"Solving the {formulation} formulation!")
    # Solve the selected model
    ampl.eval(f"solve {problem_name};")

    # Get the data from AMPL and assign them to the entities making up the grid above
    sudoku.set_values(ampl.get_data("x").to_dict())

In [39]:
sudoku = SudokuSchema(BASE)
sudoku.display()
sudoku.set_values(ampl.get_data("givenData").to_dict())

RadioButtons(description='Formulation:', layout=Layout(width='max-content'), options=('Constraint Programming'…

Solving the Constraint Programming formulation!
HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 9
1394 simplex iterations
1 branching nodes
 
Solving the MIP formulation!
HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 9
315 simplex iterations
1 branching nodes
 
